In [69]:
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import json
import re
import requests

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\thugz\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [78]:
def get_new_palette():
    driver.get("https://artsexperiments.withgoogle.com/artpalette/")

    # skip intro
    skip_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'skip-intro-btn')))

    skip_button.click()

    time.sleep(2)

    # get palette
    palette = driver.current_url.split("/")[-1]
    palette = palette.split('-')
    palette_hex_codes = ["#" + hex_code for hex_code in palette]
    palette_key = "".join(sorted(palette))

    with open("image_links.json", "r") as infile:
        palettes = json.loads(infile.read())

    if palette_key in palettes.keys():
        return null, null, null
    else:
        driver.execute_script("window.scrollTo(0, window.scrollY + 10000)")

        time.sleep(10)

        # read into beautifulsoup
        page_source = driver.page_source

        soup = BeautifulSoup(page_source, 'html.parser')

        palette_images = []

        image_pages = soup.find_all("a", {"class": "more-details"})

        for image_page in image_pages:
            current_image = {}
            # get image title, remove non-ascii
            encoded_title = image_page["title"].encode('ascii', 'ignore')
            title = encoded_title.decode()
            current_image["title"] = image_page["title"]

            # get larger image given this original link
            image_html = requests.get(image_page["href"]).content
            soup = BeautifulSoup(image_html, 'html.parser')
            image = soup.find("img")
            image_link = "https:" + image["src"]
            current_image["link"] = image_link

            palette_images.append(current_image)

        return palette_hex_codes, palette_key, palette_images


In [79]:
num_processed = 0

while num_processed < 2000:
    print(num_processed)

    palette_hex_codes, palette_key, image_dict = get_new_palette()

    if palette_key:
        with open("image_links.json") as infile:
            palettes = json.loads(infile.read())
            palettes[palette_key] = {"palette_hex_codes": palette_hex_codes, "images": image_dict}
        with open("image_links.json", "w") as outfile:
            json.dump(palettes, outfile)
        num_processed += 1

    else:
        num_processed += 1
        continue

0
b'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">\n<html>\n<head><meta http-equiv="content-type" content="text/html; charset=utf-8"><meta name="viewport" content="initial-scale=1"><title>https://artsandculture.google.com/asset/GAEwMMeAFWaY9Q?utm_campaign=cilex_v1&amp;utm_source=cilab&amp;utm_medium=artsexperiments&amp;utm_content=artpalette</title></head>\n<body style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px;" onload="e=document.getElementById(\'captcha\');if(e){e.focus();}">\n<div style="max-width:400px;">\n<hr noshade size="1" style="color:#ccc; background-color:#ccc;"><br>\n<form id="captcha-form" action="index" method="post">\n<script src="https://www.google.com/recaptcha/api.js" async defer></script>\n<script>var submitCallback = function(response) {document.getElementById(\'captcha-form\').submit();};</script>\n<div id="recaptcha" class="g-recaptcha" data-sitekey="6LfwuyUTAAAAAOAmoS0fdqijC2PbbdH4kjq62Y1b"

TypeError: 'NoneType' object is not subscriptable

In [67]:
r = requests.get("https://artsandculture.google.com/asset/7wELdw4KnbezyA?utm_campaign=cilex_v1&utm_source=cilab&utm_medium=artsexperiments&utm_content=artpalette")
    # "https://artsandculture.google.com/asset/VAGgshZL5aWXfw?utm_campaign=cilex_v1&utm_source=cilab&utm_medium=artsexperiments&utm_content=artpalette")
if r.status_code == 429:
    print("ok")

ok


In [57]:
soup = BeautifulSoup(r, 'html.parser')

# image = soup.select(".inCeke img")
images = soup.find("img")
print(images)

"https:" + images["src"]

None


TypeError: 'NoneType' object is not subscriptable